In [7]:
import sys, os
sys.path.append(os.path.realpath(".."))
os.environ["TF_XLA_FLAGS"]="--tf_xla_cpu_global_jit"

import util_funcs
from importlib import reload
import data_reader as read
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt
import constants
import ensembleReader as er
import clinical_text_analysis as cta
import tsfresh
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve
from os import path
# import predictSeizureConvExp as psce
import keras_models.dataGen as dg
from addict import Dict
from pathlib import Path
# reload(psce)
from keras.utils import multi_gpu_model
import keras
import keras.optimizers
import time
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score, log_loss

In [9]:
train_data = pkl.load(open("/n/scratch2/ms994/train_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [8]:
test_data =  pkl.load(open("/n/scratch2/ms994/test_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [3]:
valid_data = pkl.load(open("/n/scratch2/ms994/valid_multiple_labels_sessions_montage_seizure_data_4.pkl", "rb"))

In [10]:
def patient_func(tkn_file_paths, session_instead_patient):
        return [read.parse_edf_token_path_structure(tkn_file_path)[1] + "/" + read.parse_edf_token_path_structure(tkn_file_path)[2] for tkn_file_path in tkn_file_paths]

In [18]:
seizureInfo = pd.read_csv("/home/ms994/dbmi_eeg_clustering/assets/seizures.csv")

In [20]:
seizureInfo.groupby("seizure_type")["seizure_type"].count() 

seizure_type
ABSZ      99
CPSZ     361
FNSZ    1834
GNSZ     585
MYSZ       3
SPSZ      52
TCSZ      53
TNSZ      68
Name: seizure_type, dtype: int64

Use the top 3 seizures only, CPSZ, FNSZ, and GNSZ. only include a patient if they are characterized by the seizure or have no seizure in the set. It is either unfair/incorrect/too hard to include the healthy state of patients not in this subpopulation

In [32]:
noSeizureFiles = seizureInfo[pd.isnull(seizureInfo.seizure_type)].fn

In [33]:
seizurePatientMap = seizureInfo.groupby(["patient", "seizure_type"])["patient", "seizure_type"].count()

In [41]:
seizurePatientMap.columns = ["patient_cnt", "seizure_type_cnt"]

In [44]:
seizurePatientMap = seizurePatientMap.reset_index()

In [62]:
matchingPatients = seizurePatientMap[seizurePatientMap.seizure_type.isin(["CPSZ", "FNSZ", "GNSZ"])].patient

In [63]:
noSeizurepatients = [read.parse_edf_token_path_structure(file)[1] for file in noSeizureFiles]

In [74]:
matchingPatients = matchingPatients.apply(lambda a: "{:08d}".format(int(a))).tolist()

In [77]:
toFilterBy = noSeizurepatients + matchingPatients

In [52]:
"{}/{}".format(read.parse_edf_token_path_structure(noSeizureFiles[0])[1], read.parse_edf_token_path_structure(noSeizureFiles[0])[2])

'00000258/s002_2003_07_21'

In [56]:
train_data[1][1][1].split("/")[0] in no

'00013182'

In [78]:
def update_data(edss,  seizure_classes_to_use=["GNSZ", "FNSZ", "CPSZ"], include_seizure_type=True, include_montage_channels=True, remove_outlier_by_std_thresh=100, zero_center_over_time=True, is_eval_set=False):
    '''
    since we store the full string of the session or the patient instead of the index, we update the data to use the int index
    some of the tasks require different datasets and some filtering of the data i.e. only seizure classification or just some of the labels
    is_eval_set: for use with valid and test set, since they shouldn't have patients from the train set and therefore predicting for them is wrong
    '''

    data = [datum[0] for datum in edss]
    if is_eval_set:
        patient_labels = [0 for i in range(len(edss))]
        remove_outlier_by_std_thresh=None
    else:
        patients = [datum[1][1] for datum in edss]
        allPatient = list(set(patients))
        patient_labels = [allPatient.index(patient) for patient in patients]
    seizure_detection_labels = [datum[1][0] for datum in edss]
    if include_seizure_type:
        seizure_class_labels = [datum[1][2] for datum in edss]
    if include_montage_channels:
        montage_channel_labels = [datum[1][3] for datum in edss]
    keep_index = [True for i in range(len(data))]
    if seizure_classes_to_use is not None:
        for i, seizure_class_label in enumerate(seizure_class_labels):
            if seizure_detection_labels[i] and constants.SEIZURE_SUBTYPES[seizure_class_label] not in seizure_classes_to_use: #it's a seizure detection class and it should be excluded
                keep_index[i] = False
    for i, patient_session in enumerate(patients):
        if patient_session.split("/")[0] not in toFilterBy:
            keep_index[i] = False
        

    if zero_center_over_time:
        for i, datum in enumerate(data):
            data[i] = (datum.T - datum.mean(1)).T
    if remove_outlier_by_std_thresh is not None:
        removed = 0
        for i, datum in enumerate(data):
            if np.std(datum) > remove_outlier_by_std_thresh:
                keep_index[i] = False
                removed += 1
        print("We removed {} because unclean by std".format(removed))


    data_to_keep = []
    patient_labels_to_keep = []
    seizure_detect_to_keep = []
    seizure_class_labels_to_keep = []
    montage_channel_labels_to_keep = []

    for i, should_keep in enumerate(keep_index):
        if should_keep:
            data_to_keep.append(data[i])
            patient_labels_to_keep.append(patient_labels[i])
            seizure_detect_to_keep.append(seizure_detection_labels[i])
            if include_montage_channels:
                montage_channel_labels_to_keep.append(montage_channel_labels[i])
            if include_seizure_type:
                seizure_class_labels_to_keep.append(seizure_class_labels[i])
    return [(data_to_keep[i], (seizure_detect_to_keep[i], patient_labels_to_keep[i], seizure_class_labels_to_keep[i], montage_channel_labels_to_keep[i].values)) for i in range(len(data_to_keep))], seizure_detect_to_keep, patient_labels_to_keep, seizure_class_labels_to_keep, montage_channel_labels_to_keep

In [ ]:
train_data = update_data(train_data)
valid_data = update_data(valid_data)
test_data = update_data(test_data)

In [ ]:
train_data.shape